# Данный код полжен считывать техрежим и создавать модели скважин в Pipesim.



In [3]:
'''
    PIPESIM Python Toolkit
    Example: Update esp motor and cable values

'''
print("Running example file '{}'".format("C:\Program Files\Schlumberger\PIPESIM2021.1\Developer Tools\Python Toolkit\Examples\get_set_esp_motor_and_cable.py"))

from sixgill.pipesim import Model
#from utilities import open_model_from_case_study
from sixgill.definitions import ModelComponents, Parameters, Constants, SystemVariables, ProfileVariables, SimulationOptions

import tempfile

# Open the model
model = Model.open("./Models/CSW_101_Basic Oil Well.pips")
#model = open_model_from_case_study("./Well Models/CSW_101_Basic Oil Well.pips")

#

print("Adding an ESP to Well_1")
model.add(ModelComponents.ESP, "Esp1", context="Well_1", \
         parameters={Parameters.ESP.TOPMEASUREDDEPTH:8370,
                     Parameters.ESP.OPERATINGFREQUENCY:90,
                     Parameters.ESP.MANUFACTURER:"ALNAS",
                     Parameters.ESP.MODEL:"ANA580",
                     Parameters.ESP.NUMBERSTAGES:100,
                     Parameters.ESP.HEADFACTOR:1,
                     Parameters.ESP.POWERFACTOR:0.95,
                     Parameters.ESP.USEVISCOSITYCORRECTION:True,
                     Parameters.ESP.MotorCatalogData.MANUFACTURER:"M",
                     Parameters.ESP.MotorCatalogData.MOTORNAME:"M2",
                     Parameters.ESP.MotorCoefficients.AMPValues:[1, 2, 3], #ampValues for the first three coeffiecients
                     Parameters.ESP.MotorCoefficients.PFValues:[0.1],      #pfValue for the first coefficient
                     Parameters.ESP.MotorCoefficients.EFFValues:[1, 2.5]   #effValues for the first two coefficients
                     },
                     )

print('setting the effValue for the first motor coefficient')
model.set_value(context="Well_1:Esp1",
                parameter=Parameters.ESP.MotorCoefficients.EFFValues,
                value=[0.05])

print('using model.set_values to set effValues and operating frequency')
esp_motor_params = {
    "Well_1:Esp1": {
        Parameters.ESP.MotorCoefficients.EFFValues: [1, 2.5],
        Parameters.ESP.OPERATINGFREQUENCY: 90
    }
}

model.set_values(esp_motor_params)

print(
    'getting all values for Esp1 (including motor and cable and coefficient values\n'
)
parameters = model.get_values(context="Well_1:Esp1")
values_map = parameters["Well_1:Esp1"]
for key, value in values_map.items():
    print('{}:{}'.format(key, value))

print('\ngetting head factor and motor name for Esp1\n')
parameters = model.get_values(context="Well_1:Esp1",
                              parameters=[
                                  Parameters.ESP.HEADFACTOR,
                                  Parameters.ESP.MotorCatalogData.MOTORNAME
                              ])
print(parameters)

print('\ngetting head factor and ampValues for Esp1\n')
parameters = model.get_values(context="Well_1:Esp1",
                              parameters=[
                                  Parameters.ESP.HEADFACTOR,
                                  Parameters.ESP.MotorCoefficients.AMPValues
                              ])
print(parameters)

print('\ngetting head factor, effValues, maxamps for Esp1\n')
parameters = model.get_values(context="Well_1:Esp1",
                              parameters=[
                                  Parameters.ESP.HEADFACTOR,
                                  Parameters.ESP.MotorCoefficients.EFFValues,
                                  Parameters.ESP.CableCatalogData.MAXAMPS
                              ])
print(parameters)

print('\ngetting head factor for Esp1\n')
parameters = model.get_value(context="Well_1:Esp1",
                             parameter=Parameters.ESP.HEADFACTOR)
print(parameters)

print('\ngetting maxamp for Esp1\n')
parameters = model.get_value(context="Well_1:Esp1",
                             parameter=Parameters.ESP.CableCatalogData.MAXAMPS)
print(parameters)

print('\ngetting ampValues for Esp1\n')
parameters = model.get_value(
    context="Well_1:Esp1",
    parameter=Parameters.ESP.MotorCoefficients.AMPValues)
print(parameters)

print('\ngetting motor name for Esp1\n')
parameters = model.get_value(
    context="Well_1:Esp1", parameter=Parameters.ESP.MotorCatalogData.MOTORNAME)
print(parameters)

#Now add some reasonable values for motor and cable and run a simulation
esp_motor_params = {
    "Well_1:Esp1": {
        Parameters.ESP.MotorCoefficients.EFFValues:
        [-0.0155, 1.5982, -2.3615, 4.3328, -3.7083, 1.1501],
        Parameters.ESP.MotorCoefficients.EFFValues:
        [-0.0357, 3.6761, -7.9931, 9.8977, -6.4203, 1.6615],
        Parameters.ESP.MotorCoefficients.PFValues:
        [0.1608, 0.9299, -0.9556, 1.7459, -1.7823, 0.6141],
        Parameters.ESP.MotorCatalogData.MOTORNAME:
        "ESP STD",
        Parameters.ESP.MotorCatalogData.NPPOWER:
        10,
        Parameters.ESP.MotorCatalogData.NPAMPERAGE:
        15,
        Parameters.ESP.MotorCatalogData.NPVOLTAGE:
        435,
        Parameters.ESP.MotorCatalogData.FLOOREFFICIENCY:
        1,
        Parameters.ESP.MotorCatalogData.FLOORPOWERFACTOR:
        1,
        Parameters.ESP.CableCatalogData.VDROP:
        0.11,
        Parameters.ESP.CableCatalogData.NAME:
        "#3/0 Cu",
        Parameters.ESP.CABLELENGTHBELOWPUMP:
        100,
        Parameters.ESP.POWERCORRECTION:True,
        Parameters.ESP.ISPOWERCORRECTIONCALCULATED:True,
    }
}

model.set_values(esp_motor_params)

#run the pt profile simulation
sys_variables = [
    SystemVariables.PRESSURE_DIFFERENCE,  # dp
    SystemVariables.POWER,  #total power
    SystemVariables.SURFACEPOWER,  #Surface power
    SystemVariables.OUTLET_PRESSURE,  #Pout
]

profile_variables = [
    ProfileVariables.TEMPERATURE,
    ProfileVariables.ELEVATION,
    ProfileVariables.TOTAL_DISTANCE,
]

print("Running PT/Profile simulation...")
results = model.tasks.ptprofilesimulation.run(
    producer="Well_1",
    system_variables=sys_variables,
    profile_variables=profile_variables,
    study="Study 1")

case = results.cases[0]

surface_power = results.node[case][SystemVariables.SURFACEPOWER]["Esp1"]
print("Surface power for Esp1:{}".format(surface_power))

# Save and close the model
print("All finished. Saving model...")
filename = tempfile.gettempdir() + "/ExercisingAddAndUpdateEspMotor.pips"


model.save(filename)
model.open_ui()
#model.close()


INFO:manta.server.manager:Starting PIPESIM server on thread ID: 5336


Running example file 'C:\Program Files\Schlumberger\PIPESIM2021.1\Developer Tools\Python Toolkit\Examples\get_set_esp_motor_and_cable.py'


INFO:manta.server.manager:Python toolkit license is checked out successfully
INFO:manta.server.manager:Waiting for PIPESIM server to start.
INFO:sixgill.core.metadata:Using cached metadata for 'http://localhost:55468/api/metadata#'


Adding an ESP to Well_1
setting the effValue for the first motor coefficient
using model.set_values to set effValues and operating frequency
getting all values for Esp1 (including motor and cable and coefficient values

Name:Esp1
Diameter:3.62
HeadFactor:1.0
Manufacturer:ALNAS
Model:ANA580
BaseFrequency:60.0
CasingID:nan
ESPSpeedFactor:0.97222
FlowrateFactor:1.0
GasSeparatorEfficiency:100.0
HasGasSeparator:False
MaxFlowRate:868.0
MinFlowRate:453.0
NumberStages:100
OperatingFrequency:90.0
OperatingProductionRate:nan
Power:nan
PowerFactor:0.95
Series:92
SlipFactor:nan
UsePower:False
UseStageByStageCalc:True
UseViscosityCorrection:True
CoefficientA:1.0
CoefficientB:0.0
IsPowerCorrectionCalculated:True
PowerCorrection:True
CableLengthBelowPump:False
RecombineGas:100.0
IsActive:True
CableName:
VDrop:nan
MaxAmps:nan
MotorManufacturer:M
MotorName:M2
NPPower:nan
NPVoltage:nan
NPAmperage:nan
FloorEfficiency:nan
FloorPowerFactor:nan
AMPValues:[1.0, 2.0, 3.0, nan, nan, nan]
EFFValues:[1.0, 2.5, n

INFO:sixgill.core.run_or_start_simulation:Simulation is running.
INFO:sixgill.core.run_or_start_simulation:Simulation is finished.
INFO:sixgill.core.run_or_start_simulation:Simulation run successfully.


Surface power for Esp1:61.277598007446
All finished. Saving model...


In [2]:
'''
    PIPESIM Python Toolkit
    Example: Create and parameterize a well

'''
print("Running example file '{}'".format("C:\Program Files\Schlumberger\PIPESIM2021.1\Developer Tools\Python Toolkit\Examples\add_well.py"))


from sixgill.pipesim import Model
from sixgill.definitions import ModelComponents, Parameters, Constants
from utilities import create_new_model

# Create a new model
model = Model.new("./Models/new_workspace.pips", overwrite=True)
#model = create_new_model("AddWell.pips", overwrite=True)

print("Adding a black oil fluid named BK111")
model.add(ModelComponents.BLACKOILFLUID, "BK111", parameters = {Parameters.BlackOilFluid.GOR:200, Parameters.BlackOilFluid.WATERCUT:5} )

print("Adding a well named 'Well 2'.")
model.add(ModelComponents.WELL, "Well 2", parameters = {Parameters.Well.AMBIENTTEMPERATURE:60})

model.set_value(Well = "Well 2", parameter=Parameters.Well.ASSOCIATEDBLACKOILFLUID, value="BK111")

print("Setting the tubing and casing for Well 2.")
model.add(ModelComponents.CASING, "Csg1", context="Well 2", \
         parameters={Parameters.Casing.TOPMEASUREDDEPTH:0,
                     Parameters.Casing.LENGTH:1000,
                     Parameters.Casing.INNERDIAMETER:12.0,
                     Parameters.Casing.BOREHOLEDIAMETER:30,
                     Parameters.Casing.ROUGHNESS:0.001,
                     Parameters.Casing.THERMALCONDUCTIVITY:34.001,
                     Parameters.Casing.DENSITY:2.001,
                     Parameters.Casing.FLUIDTHERMALCONDUCTIVITY:2.001,
                     Parameters.Casing.CEMENTTHERMALCONDUCTIVITY:2.001,
                     Parameters.Casing.WALLTHICKNESS:0.5})

model.add(ModelComponents.CASING, "Csg2", context="Well 2", \
         parameters={Parameters.Casing.TOPMEASUREDDEPTH:0,
                     Parameters.Casing.LENGTH:3000,
                     Parameters.Casing.INNERDIAMETER:8,
                     Parameters.Casing.BOREHOLEDIAMETER:30,
                     Parameters.Casing.ROUGHNESS:0.001,
                     Parameters.Casing.WALLTHICKNESS:0.5})

model.add(ModelComponents.LINER, "Csg3", context="Well 2", \
         parameters={Parameters.Liner.TOPMEASUREDDEPTH:2950,
                     Parameters.Liner.LENGTH:1050,
                     Parameters.Liner.INNERDIAMETER:6,
                     Parameters.Liner.BOREHOLEDIAMETER:30,
                     Parameters.Liner.ROUGHNESS:0.001,
                     Parameters.Liner.WALLTHICKNESS:0.3})

model.add(ModelComponents.TUBING, "Tub1", context="Well 2", \
         parameters={Parameters.Tubing.TOPMEASUREDDEPTH:0,
                     Parameters.Tubing.LENGTH:3000,
                     Parameters.Tubing.INNERDIAMETER:4,
                     Parameters.Tubing.ROUGHNESS:0.001,
                     Parameters.Tubing.WALLTHICKNESS:0.2})

model.add(ModelComponents.TUBING, "Tub2", context="Well 2", \
         parameters={Parameters.Tubing.TOPMEASUREDDEPTH:3000,
                     Parameters.Tubing.LENGTH:1000,
                     Parameters.Tubing.INNERDIAMETER:3,
                     Parameters.Tubing.ROUGHNESS:0.001,
                     Parameters.Tubing.WALLTHICKNESS:0.2})

model.add(ModelComponents.PACKER, "Packer", context="Well 2", parameters={Parameters.Packer.TOPMEASUREDDEPTH:2000})

############################################################################
#Follow code snippets demonstrates how to add other downhole equipment
############################################################################

print("Adding a choke to Well 2")
model.add(ModelComponents.CHOKE, "CK1", context="Well 2", \
         parameters={Parameters.Packer.TOPMEASUREDDEPTH:2000,
                     Parameters.Choke.BEANSIZE:2,
                     Parameters.Choke.SUBCRITICALCORRELATION:Constants.SubCriticalFlowCorrelation.ASHFORD})

print("Adding an ESP to well 2")
model.add(ModelComponents.ESP, "Esp1", context="Well 2", \
         parameters={Parameters.ESP.TOPMEASUREDDEPTH:1500,
                     Parameters.ESP.OPERATINGFREQUENCY:60,
                     Parameters.ESP.MANUFACTURER:"ALNAS",
                     Parameters.ESP.MODEL:"ANA580",
                     Parameters.ESP.NUMBERSTAGES:100,
                     Parameters.ESP.HEADFACTOR:1,
                     Parameters.ESP.POWERFACTOR:0.95,
                     Parameters.ESP.USEVISCOSITYCORRECTION:True
                    },
                    )

#model.add(ModelComponents.PCP, "Pcp1", context="Well 2", \
#         parameters={Parameters.PCP.TOPMEASUREDDEPTH:700,
#                     Parameters.PCP.OPERATINGSPEED:100,
#                     Parameters.PCP.MANUFACTURER:"Weatherford",
#                     Parameters.PCP.MODEL:"BMW 100-4000",
#                     Parameters.PCP.ISTOPDRIVE:False,
#                     Parameters.PCP.HEADFACTOR:1,
#                     Parameters.PCP.POWERFACTOR:0.95,
#                     Parameters.PCP.FLOWRATEFACTOR:0.98,
#                     Parameters.PCP.USEVISCOSITYCORRECTION:True})

#model.add(ModelComponents.RODPUMP, "Rp1", context="Well 2", \
#         parameters={Parameters.RodPump.TOPMEASUREDDEPTH:300,
#                     Parameters.RodPump.NOMINALFLOWRATE:100,
#                     Parameters.RodPump.RODDIAMETER:1.2,
#                     Parameters.RodPump.HASGASSEPARATOR:True})

#from sixgill.definitions import GasLiftType
#model.add(ModelComponents.GASLIFTINJECTION, "Inj1", context="Well 2", \
#         parameters={Parameters.GasLiftInjection.TOPMEASUREDDEPTH:400,
#                     Parameters.GasLiftInjection.GASLIFTTYPE:GasLiftType.GLR,
#                     Parameters.GasLiftInjection.GLR:10})
#model.add(ModelComponents.GASLIFTINJECTION, "Inj2", context="Well 2", \
#         parameters={Parameters.GasLiftInjection.TOPMEASUREDDEPTH:450,
#                     Parameters.GasLiftInjection.MANUFACTURER:"Bompet", \
#                     Parameters.GasLiftInjection.SERIES:"ERO4-JR", \
#                     Parameters.GasLiftInjection.VALVETYPE:"IPO", \
#                     Parameters.GasLiftInjection.PORTSIZE:0.25, \
#                     Parameters.GasLiftInjection.DISCHARGECOEFFICIENT:65, \
#                     Parameters.GasLiftInjection.DISCHARGETOFULLYOPEN:995, \
#                     Parameters.GasLiftInjection.SPRINGPRESSURE:1000, \
#                     Parameters.GasLiftInjection.PTRO:1000, \
#                     })

#model.add(ModelComponents.ENGINEKEYWORDS, "Ek1", context="Well 2", \
#         parameters={Parameters.EngineKeywords.TOPMEASUREDDEPTH:100,
#                     Parameters.EngineKeywords.KEYWORDS:"print all"})

#model.add(ModelComponents.SLIDINGSLEEVE, "Slv1", context="Well 2", \
#         parameters={Parameters.SlidingSleeve.TOPMEASUREDDEPTH:950, Parameters.SlidingSleeve.ISOPEN:False})

#model.add(ModelComponents.SUBSURFACESAFETYVALVE, "SSV1", context="Well 2", \
#         parameters={Parameters.SubsurfaceSafetyValve.TOPMEASUREDDEPTH:250, Parameters.SubsurfaceSafetyValve.BEANID:2})

print("Adding a vertical completion to Well 2")
model.add(ModelComponents.COMPLETION, "VertComp11", context="Well 2", \
         parameters={Parameters.Completion.TOPMEASUREDDEPTH:2800,
                     Parameters.Completion.FLUIDENTRYTYPE:Constants.CompletionFluidEntry.SINGLEPOINT,
                     Parameters.Completion.GEOMETRYPROFILETYPE:Constants.Orientation.VERTICAL,
                     Parameters.Completion.IPRMODEL: Constants.IPRModels.IPRPIMODEL,
                     Parameters.Completion.RESERVOIRPRESSURE:4000,
                     Parameters.IPRPIModel.ISGASMODEL:True,
                     Parameters.IPRPIModel.GASPI:5,
                     Parameters.Completion.RESERVOIRTEMPERATURE:150,
                     Parameters.Well.ASSOCIATEDBLACKOILFLUID: "BK111" })

print("Adding a horizontal completion to Well 2")
model.add(ModelComponents.COMPLETION, "HoriComp", context="Well 2", \
         parameters={Parameters.Completion.TOPMEASUREDDEPTH:3500,
                     Parameters.Completion.BOTTOMMEASUREDDEPTH:4000,
                     Parameters.Completion.FLUIDENTRYTYPE:Constants.CompletionFluidEntry.DISTRIBUTED,
                     Parameters.Completion.GEOMETRYPROFILETYPE:Constants.Orientation.HORIZONTAL,
                     Parameters.Completion.IPRMODEL: Constants.IPRModels.IPRPSSBABUODEH,
                     Parameters.DistributedCompletionModel.ISGASMODEL:True,
                     Parameters.DistributedCompletionModel.GASPI:5,
                     Parameters.DistributedCompletionModel.RESERVOIRXDIM:1000,
                     Parameters.DistributedCompletionModel.RESERVOIRYDIM:1000,
                     Parameters.DistributedCompletionModel.RESERVOIRTHICKNESS:1000,
                     Parameters.DistributedCompletionModel.PERMEABILITYX:5,
                     Parameters.DistributedCompletionModel.PERMEABILITYY:4,
                     Parameters.DistributedCompletionModel.RESERVOIREXTENT:10,
                     Parameters.DistributedCompletionModel.LOCATIONECCEN:10,
                     Parameters.DistributedCompletionModel.COMPLETIONVERTICALPERMRATIO:0.9,  #kv/k
                     Parameters.DistributedCompletionModel.LOCATIONXPOS:4,
                     Parameters.DistributedCompletionModel.LOCATIONYPOS:1,
                     Parameters.DistributedCompletionModel.LOCATIONZPOS:17,
                     Parameters.DistributedCompletionModel.WELLLENGTH:100,
                     Parameters.Completion.RESERVOIRPRESSURE:4000,
                     Parameters.Completion.RESERVOIRTEMPERATURE:150 })

model.set_value(context="Well 2:HoriComp", parameter=Parameters.Well.ASSOCIATEDBLACKOILFLUID, value="BK111")

# Save and close the model
print("All finished. Saving model...")
model.open_ui()
#model.save()
#model.close()


INFO:manta.server.manager:Starting PIPESIM server on thread ID: 17680


Running example file 'C:\Program Files\Schlumberger\PIPESIM2021.1\Developer Tools\Python Toolkit\Examplesdd_well.py'


INFO:manta.server.manager:Python toolkit license is checked out successfully
INFO:manta.server.manager:Waiting for PIPESIM server to start.
INFO:sixgill.core.metadata:Using cached metadata for 'http://localhost:58987/api/metadata#'


Adding a black oil fluid named BK111
Adding a well named 'Well 2'.
Setting the tubing and casing for Well 2.
Adding a choke to Well 2
Adding an ESP to well 2
Adding a vertical completion to Well 2
Adding a horizontal completion to Well 2
All finished. Saving model...


In [23]:
from sixgill.pipesim import Model
from sixgill.definitions import *

# Create a new model
model = Model.new("./Models/new_workspace.pips", overwrite=True, units=Units.METRIC)
print("Model created")

print("Adding a black oil fluid named BK111")
model.add(ModelComponents.BLACKOILFLUID, "BK111",
          parameters = {Parameters.BlackOilFluid.GOR:200,
                        Parameters.BlackOilFluid.WATERCUT:5
                       } )

print("Adding a well named 'Well 2'.")
model.add(ModelComponents.WELL, "Well 2",
          parameters = {Parameters.Well.AMBIENTTEMPERATURE:60
                       })

model.set_value(Well = "Well 2", parameter=Parameters.Well.ASSOCIATEDBLACKOILFLUID, value="BK111")

model.save(view_type=ViewTypes.WELL)
model.open_ui()

INFO:manta.server.manager:Starting PIPESIM server on thread ID: 9264
INFO:manta.server.manager:Python toolkit license is checked out successfully
INFO:manta.server.manager:Waiting for PIPESIM server to start.
INFO:sixgill.core.metadata:Using cached metadata for 'http://localhost:64808/api/metadata#'


Model created
Adding a black oil fluid named BK111
Adding a well named 'Well 2'.


NameError: name 'ViewTypes' is not defined

In [15]:
Model.open_ui("./Models/new_workspace.pips")

In [19]:
model = Model.open("./Models/new_workspace.pips")

INFO:manta.server.manager:Starting PIPESIM server on thread ID: 15528
INFO:manta.server.manager:Python toolkit license is checked out successfully
INFO:manta.server.manager:Waiting for PIPESIM server to start.
INFO:sixgill.core.metadata:Using cached metadata for 'http://localhost:64785/api/metadata#'


In [16]:
model.about()

{'WebAPIVersion': '2021.1.687.0',
 'ToolkitVersion': '2021.1',
 'ModelFilename': './Models/new_workspace.pips',
 'UnitSystem': 'PIPESIM_FIELD',
 'UiUnitSystem': 'PIPESIM_METRIC'}

In [22]:
model.find()

['',
 'BK111',
 'Default',
 'PIPESIM_METRIC',
 'Well 2',
 'Well 2:Default',
 'Well 2:Well Outlet 10949']

In [24]:
ViewTypes.WELL

NameError: name 'ViewTypes' is not defined

In [30]:
model.describe("Well 2",parameter=Parameters.Well.ASSOCIATEDBLACKOILFLUID)